In [225]:
import pandas as pd
import numpy as np

 PRE-PROCESSING

In [226]:
df = pd.read_csv('E:\Study\Sem 6\capstone\code\Digital-Twin-in-python-main\discharge.csv')

df=df[df.Battery=='B0005']

# Create a new column with index values
df['index'] = df.index
df.set_index('index',inplace=True)

In [227]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45122 entries, 0 to 45121
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Voltage_measured      45122 non-null  float64
 1   Current_measured      45122 non-null  float64
 2   Temperature_measured  45122 non-null  float64
 3   Current_charge        45122 non-null  float64
 4   Voltage_charge        45122 non-null  float64
 5   Time                  45122 non-null  float64
 6   Capacity              45122 non-null  float64
 7   id_cycle              45122 non-null  int64  
 8   type                  45122 non-null  object 
 9   ambient_temperature   45122 non-null  int64  
 10  time                  45122 non-null  float64
 11  Battery               45122 non-null  object 
dtypes: float64(8), int64(2), object(2)
memory usage: 4.5+ MB


In [228]:
df=df.drop(axis=1,columns=['type','ambient_temperature','time','Battery'])
#df.info()

In [229]:
df=df[['Voltage_measured','Current_measured','Temperature_measured','Current_charge','Voltage_charge','Capacity','id_cycle','Time']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45122 entries, 0 to 45121
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Voltage_measured      45122 non-null  float64
 1   Current_measured      45122 non-null  float64
 2   Temperature_measured  45122 non-null  float64
 3   Current_charge        45122 non-null  float64
 4   Voltage_charge        45122 non-null  float64
 5   Capacity              45122 non-null  float64
 6   id_cycle              45122 non-null  int64  
 7   Time                  45122 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 3.1 MB


In [230]:
cycles=df['id_cycle'].unique()
cycles.shape[0]

168

In [231]:
#HAVEN'T USED YET , WILL BE BETTER IF DONE SO 
temp_ranges=np.array([[23,26.5],[26.5,30],[30,33],[33,36],[36,37],[37,38],[38,39],[39,41]])
temp_ranges.shape

(8, 2)

In [232]:
intervals=10      # NO. OF TIME INTERVALS TAKEN PER CYCLE  
for cycle in cycles:
    min_time=df.loc[(df['id_cycle']==cycle),'Time'].min(axis=0)
    max_time=df.loc[(df['id_cycle']==cycle),'Time'].max(axis=0)
    interval_length=(max_time-min_time)/intervals
    for i in range((intervals)):
        lower_bound=min_time+i*(interval_length)
        upper_bound=min_time+(i+1)*(interval_length)
        
        average=df.loc[(df['Time']>lower_bound) & (df['Time']<=upper_bound) & (df['id_cycle']==cycle),df.columns != 'Time'].mean(axis=0)
        average=np.array(average)
        average=np.append(average,[upper_bound])

        df.loc[(df['Time']>lower_bound) & (df['Time']<=upper_bound) & (df['id_cycle']==cycle),:]=average

        indexes_to_remove=(df.loc[(df['Time']>lower_bound) & (df['Time']<=upper_bound) & (df['id_cycle']==cycle)]==True).index
        df.drop(indexes_to_remove[1:],inplace=True)

        df = df.reset_index()
        df['index'] = df.index
        df=df.set_index('index')
        

In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1856 entries, 0 to 1855
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Voltage_measured      1856 non-null   float64
 1   Current_measured      1856 non-null   float64
 2   Temperature_measured  1856 non-null   float64
 3   Current_charge        1856 non-null   float64
 4   Voltage_charge        1856 non-null   float64
 5   Capacity              1856 non-null   float64
 6   id_cycle              1856 non-null   int64  
 7   Time                  1856 non-null   float64
dtypes: float64(7), int64(1)
memory usage: 130.5 KB


In [234]:
df.loc[(df.id_cycle==11),:]

,Voltage_measured,Current_measured,Temperature_measured,Current_charge,Voltage_charge,Capacity,id_cycle,Time
index,,,,,,,,
111,3.986599,-2.011520,24.479221,1.998200,3.059000,1.82462,11,35.594
112,3.888619,-2.012234,26.021747,1.998200,2.942000,1.82462,11,361.058
113,3.786932,-2.012527,28.620490,1.998200,2.844500,1.82462,11,686.522
114,3.707863,-2.012660,29.944964,1.998211,2.768444,1.82462,11,1011.986
115,3.640570,-2.012291,30.802369,1.998222,2.702944,1.82462,11,1337.450
116,3.583420,-2.011888,31.713883,1.998259,2.647059,1.82462,11,1662.914
117,3.535766,-2.012606,32.538499,1.998256,2.599444,1.82462,11,1988.378
118,3.498748,-2.012620,33.360991,1.998224,2.561941,1.82462,11,2313.842
119,3.463713,-2.012079,34.304291,1.998271,2.526765,1.82462,11,2639.306


MODEL TRAINING

In [235]:
x=df.loc[:,df.columns!='Capacity']
y=df['Capacity']

In [236]:
x=np.array(x)
print(x.shape)
y=np.array(y)
print(y.shape)

(1856, 7)
(1856,)


In [237]:
x=x.reshape(1,x.shape[0],x.shape[1])
print(x.shape)
y=y.reshape(1,y.shape[0],1)
print(y.shape)

(1, 1856, 7)
(1, 1856, 1)


In [238]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import SGD

In [239]:
model = Sequential()
model.add(LSTM(20, input_shape=(x.shape[1],x.shape[2]),return_sequences=True))  #unroll=True
model.add(LSTM(20, input_shape=(x.shape[1],x.shape[2]),return_sequences=True))
model.add((Dense(1)))

In [240]:
algorithm=tf.keras.optimizers.Adam(learning_rate=0.001)
#algorithm = Adam(learning_rate=0.001)
model.compile(optimizer=algorithm, loss= 'mse',metrics=['mse','mae']) 

In [241]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 1856, 20)          2240      
                                                                 
 lstm_9 (LSTM)               (None, 1856, 20)          3280      
                                                                 
 dense_4 (Dense)             (None, 1856, 1)           21        
                                                                 
Total params: 5,541
Trainable params: 5,541
Non-trainable params: 0
_________________________________________________________________


In [242]:
model.fit(x,y,batch_size=1,epochs=100)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 2.2266 - mse: 2.2266 - mae: 1.4843
Epoch 2/100
1/1 [==============================] - 0s 487ms/step - loss: 2.0301 - mse: 2.0301 - mae: 1.4165
Epoch 3/100
1/1 [==============================] - 1s 521ms/step - loss: 1.8426 - mse: 1.8426 - mae: 1.3486
Epoch 4/100
1/1 [==============================] - 1s 525ms/step - loss: 1.6655 - mse: 1.6655 - mae: 1.2811
Epoch 5/100
1/1 [==============================] - 1s 550ms/step - loss: 1.5014 - mse: 1.5014 - mae: 1.2151
Epoch 6/100
1/1 [==============================] - 0s 462ms/step - loss: 1.3507 - mse: 1.3507 - mae: 1.1511
Epoch 7/100
1/1 [==============================] - 0s 449ms/step - loss: 1.2124 - mse: 1.2124 - mae: 1.0889
Epoch 8/100
1/1 [==============================] - 0s 467ms/step - loss: 1.0868 - mse: 1.0868 - mae: 1.0290
Epoch 9/100
1/1 [==============================] - 0s 479ms/step - loss: 0.9748 - mse: 0.9748 - mae: 0.9720
Epoch 10/100
1/1 [=============

In [243]:
predictions = model.predict(x)
print(predictions.shape)
predictions

1/1 [==============================] - 1s 611ms/step
(1, 1856, 1)


array([[[0.47466186],
        [0.72958946],
        [0.97662383],
        ...,
        [1.3730595 ],
        [1.3749626 ],
        [1.3761535 ]]], dtype=float32)

In [244]:
predictions=predictions.reshape(-1)
predictions.shape

(1856,)

In [245]:
from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [246]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, 
                         y=predictions,
                         mode='lines',
                         name='Modelled capacity',
                         line=dict(color='navy', 
                                   width=2.5,
                                   )))

fig.add_trace(go.Scatter(x=df.index, 
                         y=df['Capacity'],
                         mode='markers',
                         marker=dict(
                              size=4,
                              color='grey',
                              symbol='cross'
                                 ),
                         name='NASA dataset',
                         line_color='navy'))
fig.update_layout(
    title="LSTM MODEL COMPARISON FOR TRAINING DATASET",
    xaxis_title="Cycles",
    yaxis_title="𝐶, Capacity [Ahr]")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.8
))

fig.update_layout({'plot_bgcolor': '#f2f8fd',
                  'paper_bgcolor': 'white',}, 
                   template='plotly_white')

## MODEL EVALUATION ON ANOTHER BATTERY(B0006)

In [247]:
test = pd.read_csv('E:\Study\Sem 6\capstone\code\Digital-Twin-in-python-main\discharge.csv')

test=test[test.Battery=='B0006']

# Create a new column with index values
test['index'] = test.index
test.set_index('index',inplace=True)

In [248]:
test=test.drop(axis=1,columns=['type','ambient_temperature','time','Battery'])

In [249]:
test=test[['Voltage_measured','Current_measured','Temperature_measured','Current_charge','Voltage_charge','Capacity','id_cycle','Time']]
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44354 entries, 45122 to 89475
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Voltage_measured      44354 non-null  float64
 1   Current_measured      44354 non-null  float64
 2   Temperature_measured  44354 non-null  float64
 3   Current_charge        44354 non-null  float64
 4   Voltage_charge        44354 non-null  float64
 5   Capacity              44354 non-null  float64
 6   id_cycle              44354 non-null  int64  
 7   Time                  44354 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 3.0 MB


In [250]:
cycles=test['id_cycle'].unique()
cycles.shape[0]

168

In [251]:
intervals=10
for cycle in cycles:
    min_time=test.loc[(test['id_cycle']==cycle),'Time'].min(axis=0)
    max_time=test.loc[(test['id_cycle']==cycle),'Time'].max(axis=0)
    interval_length=(max_time-min_time)/intervals
    for i in range((intervals)):
        lower_bound=min_time+i*(interval_length)
        upper_bound=min_time+(i+1)*(interval_length)
        
        average=test.loc[(test['Time']>lower_bound) & (test['Time']<=upper_bound) & (test['id_cycle']==cycle),test.columns != 'Time'].mean(axis=0)
        average=np.array(average)
        average=np.append(average,[upper_bound])

        test.loc[(test['Time']>lower_bound) & (test['Time']<=upper_bound) & (test['id_cycle']==cycle),:]=average

        indexes_to_remove=(test.loc[(test['Time']>lower_bound) & (test['Time']<=upper_bound) & (test['id_cycle']==cycle)]==True).index
        test.drop(indexes_to_remove[1:],inplace=True)

test = test.reset_index()
test['index'] = test.index
test=test.set_index('index')        

In [252]:
x_test=test.loc[:,test.columns!='Capacity']
y_test=test['Capacity']

In [253]:
x_test=np.array(x_test)
print(x_test.shape)
y_test=np.array(y_test)
print(y_test.shape)

x_test=x_test.reshape(1,x_test.shape[0],x_test.shape[1])
print(x_test.shape)
y_test=y_test.reshape(1,y_test.shape[0],1)
print(y_test.shape)

(1855, 7)
(1855,)
(1, 1855, 7)
(1, 1855, 1)


In [254]:
loss=model.evaluate(x_test,y_test)

ValueError: in user code:

    File "e:\Real Softwares\Python\lib\site-packages\keras\engine\training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "e:\Real Softwares\Python\lib\site-packages\keras\engine\training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "e:\Real Softwares\Python\lib\site-packages\keras\engine\training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "e:\Real Softwares\Python\lib\site-packages\keras\engine\training.py", line 1788, in test_step
        y_pred = self(x, training=False)
    File "e:\Real Softwares\Python\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "e:\Real Softwares\Python\lib\site-packages\keras\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of

In [255]:
predictions=model.predict(x_test)
predictions.shape

1/1 [==============================] - 1s 813ms/step


(1, 1855, 1)

In [256]:
predictions=predictions.reshape(-1)
predictions.shape

(1855,)

In [257]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=test.index, 
                         y=predictions,
                         mode='lines',
                         name='Modelled capacity',
                         line=dict(color='navy', 
                                   width=2.5,
                                   )))

fig.add_trace(go.Scatter(x=test.index, 
                         y=test['Capacity'],
                         mode='markers',
                         marker=dict(
                              size=4,
                              color='grey',
                              symbol='cross'
                                 ),
                         name='NASA dataset',
                         line_color='navy'))
fig.update_layout(
    title="LSTM MODEL COMPARISON FOR TEST DATASET",
    xaxis_title="Cycles",
    yaxis_title="𝐶, Capacity [Ahr]")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.8
))

fig.update_layout({'plot_bgcolor': '#f2f8fd',
                  'paper_bgcolor': 'white',}, 
                   template='plotly_white')